# Python + K-Space Playground
Hands-on mini-lab for MRI students learning basic Python *and* how k-space builds an image.
— **No prior coding assumed** —

### Learning goals
1. **Python essentials**: variables, arrays, loops, plotting.
2. **K-space intuition**: see how a 2-D Fourier Transform turns frequency data into an image.
3. Play with undersampling and watch aliasing appear in real time.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
%matplotlib inline
plt.rcParams['figure.figsize'] = (5,4)

## 1 ▸ 3-minute Python crash
Run each cell, tweak the code, and observe the output.

In [ ]:
a = 3
b = 4
c = a**2 + b**2
print('c =', c)  # Pythagorean demo

In [ ]:
t = np.linspace(0, 1, 500)
f = 5
signal = np.sin(2*np.pi*f*t)
plt.plot(t, signal)
plt.title('5 Hz sine wave'); plt.xlabel('time (s)'); plt.ylabel('amplitude');

In [ ]:
def sine_demo(freq=5):
    t = np.linspace(0,1,500)
    sig = np.sin(2*np.pi*freq*t)
    plt.plot(t, sig)
    plt.xlabel('time(s)'); plt.ylabel('amp');
    plt.title(f'{freq} Hz sine wave'); plt.show()

interact(sine_demo, freq=IntSlider(5, min=1, max=20, step=1));

---
## 2 ▸ What is **k-space**?
Think of the sine wave above. Frequency (k) lives in a different world than time (or position).  
In 2-D MRI we record a grid of frequencies (k_x, k_y). A 2-D Fourier Transform turns that grid into the spatial image we recognise.

### 2.1 Create a simple phantom image (circle)

In [ ]:
def make_circle(N=128, radius=0.3):
    y,x = np.indices((N,N))
    cx = cy = N//2
    r = np.sqrt((x-cx)**2 + (y-cy)**2)/N
    return (r < radius).astype(float)

phantom = make_circle(128)
plt.imshow(phantom, cmap='gray'); plt.title('Object (phantom)'); plt.axis('off');

### 2.2 Compute its k-space

In [ ]:
kspace = np.fft.fftshift(np.fft.fft2(phantom))
plt.imshow(np.log(np.abs(kspace)+1e-3), cmap='magma')
plt.title('Magnitude of k-space (log scale)'); plt.axis('off');

### 2.3 Reconstruct by inverse FFT

In [ ]:
recon = np.abs(np.fft.ifft2(np.fft.ifftshift(kspace)))
fig,ax = plt.subplots(1,2,figsize=(8,4))
ax[0].imshow(phantom, cmap='gray'); ax[0].set_title('Original'); ax[0].axis('off')
ax[1].imshow(recon, cmap='gray');  ax[1].set_title('Reconstruction'); ax[1].axis('off')

## 3 ▸ Undersampling experiment
Slide the *fraction* of k-space lines we keep. Central lines = low frequencies ↔ overall shape; edges = fine detail.

In [ ]:
def undersample_demo(center_frac=0.3):
    N = phantom.shape[0]
    width = int(center_frac * N / 2)
    mask = np.zeros_like(kspace, dtype=bool)
    mask[N//2-width:N//2+width, :] = True  # keep central lines
    k_us = np.where(mask, kspace, 0)
    recon = np.abs(np.fft.ifft2(np.fft.ifftshift(k_us)))
    fig,ax = plt.subplots(1,3,figsize=(12,4))
    ax[0].imshow(np.log(np.abs(k_us)+1e-3), cmap='magma'); ax[0].set_title('Sampled k-space'); ax[0].axis('off')
    ax[1].imshow(recon, cmap='gray'); ax[1].set_title('Recon'); ax[1].axis('off')
    ax[2].imshow(phantom, cmap='gray'); ax[2].set_title('Ground truth'); ax[2].axis('off')
    plt.show()

interact(undersample_demo, center_frac=FloatSlider(0.3, min=0.05, max=1.0, step=0.05, description='center %'));

### 4 ▸ Challenges
1. Modify `make_circle` to add a second smaller circle off-centre. How does that change k-space?
2. In the undersampling demo change the mask from central *lines* to a central *square*.
3. Try adding random noise to k-space before reconstruction. What happens to the image?